# Example of using ChatGPT API for article summarization
Refer to: https://colab.research.google.com/drive/15jh4v_TBPsTyIBhi0Fz46gEkjvhzGaBR?usp=sharing


In [2]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

Note: using Google CoLab


In [3]:
# install required packages
!pip install openai
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.0/375.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 623.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/

In [4]:
# import the packages
import openai
import gradio as gr
import json
from typing import List, Dict, Tuple

client = openai.OpenAI()

# Check if you have set your ChatGPT API successfully
# You should see "Set ChatGPT API sucessfully!!" if nothing goes wrong.
try:
    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages = [{'role':'user','content': "test"}],
            max_tokens=1,
    )
    print("Set ChatGPT API sucessfully!!")
except:
    print("There seems to be something wrong with your ChatGPT API. Please follow our demonstration in the slide to get a correct one.")

Set ChatGPT API sucessfully!!


In [10]:
# Article Summarization

prompt_for_summarization = ""

# function to reset the conversation
def reset() -> List:
    global chat_history
    chat_history = []
    return []

# Initialize an empty list to store conversation history
chat_history = []

# function to call the model to generate
def interact_summarization(prompt: str, article: str, temp=1.0) -> List[Tuple[str, str]]:
    '''
    * Arguments
      - prompt: the prompt that we use in this section
      - article: the article to be summarized
      - temp: the temperature parameter of this model. Temperature is used to control the output of the chatbot.
              The higher the temperature is, the more creative response you will get.
    '''
    # Prepare messages list to include the conversation history
    messages = []

    # Include the previous conversation stored in 'chat_history'
    for input_text, response_text in chat_history:
        messages.append({'role': 'user', 'content': input_text})
        messages.append({'role': 'assistant', 'content': response_text})

    # Append the new user input to the messages
    user_input = f"{prompt}\n{article}"
    messages.append({'role': 'user', 'content': user_input})

    # Call the ChatGPT API
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temp,
        max_tokens=200,
    )

    # Get the response content
    assistant_response = response.choices[0].message.content

    # Append the new interaction to the chat history
    chat_history.append((user_input, assistant_response))

    # Return the updated chat history for display in the Gradio chatbot component
    return chat_history

# This part generates the Gradio UI interface
with gr.Blocks() as demo:
    gr.Markdown("# Part4: Chatbot!!")

    # Create a Gradio Chatbot component
    chatbot = gr.Chatbot()

    prompt_textbox = gr.Textbox(label="Prompt", value=prompt_for_summarization, visible=False)
    article_textbox = gr.Textbox(label="Input", interactive=True, value="Hi I am Deep Shah")

    with gr.Column():
        gr.Markdown("#  Temperature\nTemperature is used to control the output of the chatbot. The higher the temperature is, the more creative response you will get.")
        temperature_slider = gr.Slider(0.0, 2.0, 1.0, step=0.1, label="Temperature")

    with gr.Row():
        send_button = gr.Button(value="Send")
        reset_button = gr.Button(value="Reset")

    # Define the click event for the Send button
    send_button.click(interact_summarization, inputs=[prompt_textbox, article_textbox, temperature_slider], outputs=[chatbot])

    # Define the click event for the Reset button to clear the chat history and UI
    reset_button.click(lambda: reset(), outputs=[chatbot])

demo.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://a188e2e2021748ed05.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a188e2e2021748ed05.gradio.live
